In [1]:
import sqlite3
import pandas as pd

In [6]:
dados_mortalidade = pd.read_csv('../dados/base_dados/tabua_concatenada.csv')
arima_ets = pd.read_csv('../dados/arima-ets/tabuas_combinad.csv')
lc = pd.read_csv('../dados/lc-lm/tabua_lc.csv')
lm = pd.read_csv('../dados/lc-lm/tabua_lm.csv')

In [8]:
arima_ets['sexo'] = arima_ets['sexo'].map({'Homens': 'Masculino', 'Mulheres': 'Feminino', 'Ambos': 'Ambos'})

In [17]:
# padrão = ['ano', 'faixa_etaria', 'nMx', 'nqx', 'nAx', 'lx', 'ndx', 'nLx', 'Tx', 'ex', 'local', 'sexo']
# renomenado as colunas para o padrao
arima_ets = arima_ets.rename(columns={
    'ano': 'ano',
    'faixa_etaria': 'faixa_etaria',
    'nMx': 'nMx',
    'nqx': 'nqx',
    'ax': 'nAx',
    'lx': 'lx',
    'dx': 'ndx',
    'Lx': 'nLx',
    'Tx': 'Tx',
    'ex': 'ex',
    'local': 'local',
    'sexo': 'sexo'
})

In [20]:
print(arima_ets.columns,
dados_mortalidade.columns)

Index(['ano', 'faixa_etaria', 'nMx', 'local', 'sexo', 'nAx', 'n', 'nqx', 'lx',
       'ndx', 'nLx', 'Tx', 'ex'],
      dtype='object') Index(['ano', 'faixa_etaria', 'nMx', 'nqx', 'nAx', 'lx', 'ndx', 'nLx', 'Tx',
       'ex', 'local', 'sexo'],
      dtype='object')


In [ ]:
# --- 2. CONEXÃO COM O BANCO (Cria o arquivo .db automaticamente) ---
conn = sqlite3.connect('banco_atuarial.db')
cursor = conn.cursor()

# --- 3. CRIAR A ESTRUTURA (SQL) ---
# Habilitar chaves estrangeiras no SQLite
cursor.execute("PRAGMA foreign_keys = ON;")
# ano	faixa_etaria	nMx	nqx	nAx	lx	ndx	nLx	Tx	ex	local	sexo
# Criar tabelas
script_sql = """
CREATE TABLE IF NOT EXISTS dim_locais (id_local INTEGER PRIMARY KEY AUTOINCREMENT, nome_local TEXT UNIQUE);
CREATE TABLE IF NOT EXISTS dim_faixas (id_faixa INTEGER PRIMARY KEY AUTOINCREMENT, descricao TEXT UNIQUE);
CREATE TABLE IF NOT EXISTS dim_sexo (id_sexo INTEGER PRIMARY KEY AUTOINCREMENT, descricao TEXT UNIQUE);

CREATE TABLE IF NOT EXISTS tabua_mortalidade (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_local INTEGER,
    id_faixa INTEGER,
    id_sexo INTEGER,
    ano INTEGER,
    nMx REAL,
    nqx REAL,
    nAx REAL,
    lx REAL,
    ndx REAL,
    nLx REAL,
    Tx REAL,
    ex REAL,
    FOREIGN KEY(id_local) REFERENCES dim_locais(id_local),
    FOREIGN KEY(id_faixa) REFERENCES dim_faixas(id_faixa),
    FOREIGN KEY(id_sexo) REFERENCES dim_sexo(id_sexo)
);

CREATE TABLE IF NOT EXISTS tabua_arima_ets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_local INTEGER,
    id_faixa INTEGER,
    id_sexo INTEGER,
    ano INTEGER,
    nMx REAL,
    nqx REAL,
    nAx REAL,
    lx REAL,
    ndx REAL,
    nLx REAL,
    Tx REAL,
    ex REAL,
    FOREIGN KEY(id_local) REFERENCES dim_locais(id_local),
    FOREIGN KEY(id_faixa) REFERENCES dim_faixas(id_faixa),
    FOREIGN KEY(id_sexo) REFERENCES dim_sexo(id_sexo)
);
"""
cursor.executescript(script_sql)

In [15]:
# --- 4. POPULAR DIMENSÕES (A Mágica do Python + SQL) ---

def popular_dimensao(df, nome_tabela, coluna_df, coluna_banco):
    # Pega valores únicos do DataFrame
    unicos = df[coluna_df].unique()
    
    # Insere no banco (IGNORANDO duplicatas se já existirem)
    for item in unicos:
        cursor.execute(f"INSERT OR IGNORE INTO {nome_tabela} ({coluna_banco}) VALUES (?)", (item,))
    conn.commit()
    
    # Lê de volta do banco para pegar os IDs gerados
    return pd.read_sql(f"SELECT * FROM {nome_tabela}", conn)

# Executa para as 3 dimensões
df_dim_locais = popular_dimensao(dados_mortalidade, 'dim_locais', 'local', 'nome_local')
df_dim_faixas = popular_dimensao(dados_mortalidade, 'dim_faixas', 'faixa_etaria', 'descricao')
df_dim_sexo   = popular_dimensao(dados_mortalidade, 'dim_sexo', 'sexo', 'descricao')

In [9]:
print(df_dim_sexo)

   id_sexo  descricao
0        1      Ambos
1        2   Feminino
2        3  Masculino


In [16]:
# --- 5. PREPARAR A TABELA FATO ---
# Agora substituímos os textos pelos IDs recém criados
df_fato = dados_mortalidade.copy()

# Faz o "Merge" (Join) para trazer os IDs para o DataFrame principal
df_fato = df_fato.merge(df_dim_locais, left_on='local', right_on='nome_local')
df_fato = df_fato.merge(df_dim_faixas, left_on='faixa_etaria', right_on='descricao')
df_fato = df_fato.merge(df_dim_sexo, left_on='sexo', right_on='descricao')

# Seleciona apenas as colunas finais
cols_finais = ['id_local', 'id_faixa', 'id_sexo', 'ano', 'nMx', 'nqx', 'nAx', 'lx', 'ndx', 'nLx', 'Tx', 'ex']
df_fato_final = df_fato[cols_finais]

In [17]:
# --- 6. SALVAR NO BANCO ---
df_fato_final.to_sql('fato_projecoes', conn, if_exists='append', index=False)

print("\n--- Sucesso! Dados salvos no SQLite ---")
print("Veja como ficou a tabela fato (só números):")
print(pd.read_sql("SELECT * FROM fato_projecoes", conn))

conn.close()


--- Sucesso! Dados salvos no SQLite ---
Veja como ficou a tabela fato (só números):
          id  id_local  id_faixa  id_sexo   ano       nMx       nqx       nAx  \
0          1         1         1        1  2000  0.028834  0.028127  0.128739   
1          2         1         2        1  2000   0.00115  0.004588  1.524332   
2          3         1         3        1  2000  0.000376  0.001881  2.286499   
3          4         1         4        1  2000  0.000436  0.002178  2.751711   
4          5         1         5        1  2000  0.001266  0.006311  2.800300   
...      ...       ...       ...      ...   ...       ...       ...       ...   
47515  47516        33        16        3  2023  0.025749  0.121349  2.632319   
47516  47517        33        17        3  2023  0.042221  0.191909  2.630570   
47517  47518        33        18        3  2023  0.073506  0.311504  2.553266   
47518  47519        33        19        3  2023  0.113711  0.444458  2.544557   
47519  47520        33  

In [ ]:
# --- 2. CONEXÃO COM O BANCO (Cria o arquivo .db automaticamente) ---
conn = sqlite3.connect('banco_atuarial.db')
cursor = conn.cursor()

# --- 3. CRIAR A ESTRUTURA (SQL) ---
# Habilitar chaves estrangeiras no SQLite
cursor.execute("PRAGMA foreign_keys = ON;")

# Criar tabelas
script_sql = """
CREATE TABLE IF NOT EXISTS dim_locais (id_local INTEGER PRIMARY KEY AUTOINCREMENT, nome_local TEXT UNIQUE);
CREATE TABLE IF NOT EXISTS dim_faixas (id_faixa INTEGER PRIMARY KEY AUTOINCREMENT, descricao TEXT UNIQUE);
CREATE TABLE IF NOT EXISTS dim_sexo (id_sexo INTEGER PRIMARY KEY AUTOINCREMENT, descricao TEXT UNIQUE);

CREATE TABLE IF NOT EXISTS fato_projecoes (
    id_projecao INTEGER PRIMARY KEY AUTOINCREMENT,
    id_local INTEGER,
    id_faixa INTEGER,
    id_sexo INTEGER,
    ano INTEGER,
    previsto REAL,
    lmt_inf REAL,
    lmt_max REAL,
    FOREIGN KEY(id_local) REFERENCES dim_locais(id_local),
    FOREIGN KEY(id_faixa) REFERENCES dim_faixas(id_faixa),
    FOREIGN KEY(id_sexo) REFERENCES dim_sexo(id_sexo)
);
"""
cursor.executescript(script_sql)

# --- 4. POPULAR DIMENSÕES (A Mágica do Python + SQL) ---

def popular_dimensao(nome_tabela, coluna_df, coluna_banco):
    # Pega valores únicos do DataFrame
    unicos = df_origem[coluna_df].unique()
    
    # Insere no banco (IGNORANDO duplicatas se já existirem)
    for item in unicos:
        cursor.execute(f"INSERT OR IGNORE INTO {nome_tabela} ({coluna_banco}) VALUES (?)", (item,))
    conn.commit()
    
    # Lê de volta do banco para pegar os IDs gerados
    return pd.read_sql(f"SELECT * FROM {nome_tabela}", conn)

# Executa para as 3 dimensões
df_dim_locais = popular_dimensao('dim_locais', 'Local', 'nome_local')
df_dim_faixas = popular_dimensao('dim_faixas', 'faixa_etaria', 'descricao')
df_dim_sexo   = popular_dimensao('dim_sexo', 'sexo', 'descricao')

# --- 5. PREPARAR A TABELA FATO ---
# Agora substituímos os textos pelos IDs recém criados
df_fato = df_origem.copy()

# Faz o "Merge" (Join) para trazer os IDs para o DataFrame principal
df_fato = df_fato.merge(df_dim_locais, left_on='Local', right_on='nome_local')
df_fato = df_fato.merge(df_dim_faixas, left_on='faixa_etaria', right_on='descricao')
df_fato = df_fato.merge(df_dim_sexo, left_on='sexo', right_on='descricao')

# Seleciona apenas as colunas finais
cols_finais = ['id_local', 'id_faixa', 'id_sexo', 'ano', 'previsto', 'lmt_inf', 'lmt_max']
df_fato_final = df_fato[cols_finais]

# --- 6. SALVAR NO BANCO ---
df_fato_final.to_sql('fato_projecoes', conn, if_exists='append', index=False)

print("\n--- Sucesso! Dados salvos no SQLite ---")
print("Veja como ficou a tabela fato (só números):")
print(pd.read_sql("SELECT * FROM fato_projecoes", conn))

conn.close()